In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate, add, BatchNormalization, Activation
import os
import glob
import tifffile as tiff
# import numpy as np
# import os
# import matplotlib.pyplot as plt
# from sklearn.metrics import precision_score, recall_score, f1_score
from PIL import Image
import rasterio
from sklearn.model_selection import train_test_split;
import re
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

c:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
def conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)

    return conv2

In [5]:
def residual_conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)
        
    #skip connection    
    shortcut = Conv2D(filters, kernel_size=(1, 1), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        shortcut = BatchNormalization(axis=3)(shortcut)
    shortcut = Activation("relu")(shortcut)
    respath = add([shortcut, conv2])       
    return respath

In [6]:
def dense_block(inputs, num_filters):
    conv1 = conv_block(inputs, num_filters)
    concat = Concatenate()([inputs, conv1])
    return concat

In [7]:
def residual_unet(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = residual_conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = residual_conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = residual_conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = residual_conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
def dense_unet(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = dense_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = dense_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = dense_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = dense_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [3]:
def load_data(input_dir, mask_dir):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', os.path.basename(value))
        return int(numbers[-1]) if numbers else -1

    # Ensure the directories exist
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return None, None
    
    if not os.path.exists(mask_dir):
        print(f"Mask directory {mask_dir} does not exist.")
        return None, None

    # Search for .tif files in the directories and sort them numerically
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')), key=numerical_sort)
    mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.tif')), key=numerical_sort)

    print("Found input files:", input_files)
    print("Found mask files:", mask_files)

    if not input_files:
        print(f"No input files found in {input_dir}.")
        return None, None

    if not mask_files:
        print(f"No mask files found in {mask_dir}.")
        return None, None

    images = []
    masks = []

    for input_file, mask_file in zip(input_files, mask_files):
        # Read input file
        with rasterio.open(input_file) as src:
            img = src.read(1)  # Read the first band assuming it's a single-band image
            # print(f"Reading image from {input_file}")
            images.append(img)

        # Read mask file
        with rasterio.open(mask_file) as src:
            msk = src.read(1)  # Read the first band assuming it's a single-band image
            # print(msk)
            masks.append(msk)

    return np.array(images), np.array(masks)


In [4]:
def load_data_new(input_dir, mask_dir):
    
    def numerical_sort(value):
        numbers = re.findall(r'\d+', os.path.basename(value))
        return int(numbers[-1]) if numbers else -1

    # Ensure the directories exist
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return None, None
    
    if not os.path.exists(mask_dir):
        print(f"Mask directory {mask_dir} does not exist.")
        return None, None

    # Search for .tif files in the directories
    input_files = glob.glob(os.path.join(input_dir, '*.tif'))
    mask_files = glob.glob(os.path.join(mask_dir, '*.tif'))

    print("Found input files:", input_files)
    print("Found mask files:", mask_files)

    if not input_files:
        print(f"No input files found in {input_dir}.")
        return None, None

    if not mask_files:
        print(f"No mask files found in {mask_dir}.")
        return None, None

    images = []
    masks = []

    for mask_file in mask_files:
        # Extract the number i from the mask filename
        match = re.search(r'NDWI_Mask_(\d+)_resized_corrupt.tif', os.path.basename(mask_file))
        if match:
            i = match.group(1)
            input_file = os.path.join(input_dir, f'{i}.tif')
            
            # Check if the corresponding input file exists
            if os.path.exists(input_file):
                # Read input file
                with rasterio.open(input_file) as src:
                    img = src.read(1)  # Read the first band assuming it's a single-band image
                    images.append(img)

                # Read mask file
                with rasterio.open(mask_file) as src:
                    msk = src.read(1)  # Read the first band assuming it's a single-band image
                    masks.append(msk)
            else:
                print(f"Corresponding input file {input_file} for mask {mask_file} not found.")

    if not images or not masks:
        print("No matching pairs of images and masks found.")
        return None, None

    return np.array(images), np.array(masks)

In [ ]:
# def load_data_test_new(input_dir, mask_dir):
    
#     def numerical_sort(value):
#         numbers = re.findall(r'\d+', os.path.basename(value))
#         return int(numbers[-1]) if numbers else -1

#     # Ensure the directories exist
#     if not os.path.exists(input_dir):
#         print(f"Input directory {input_dir} does not exist.")
#         return None, None
    
#     if not os.path.exists(mask_dir):
#         print(f"Mask directory {mask_dir} does not exist.")
#         return None, None

#     # Search for .tif files in the directories
#     input_files = glob.glob(os.path.join(input_dir, '*.tif'))
#     mask_files = glob.glob(os.path.join(mask_dir, '*.tif'))

#     print("Found input files:", input_files)
#     print("Found mask files:", mask_files)

#     if not input_files:
#         print(f"No input files found in {input_dir}.")
#         return None, None

#     if not mask_files:
#         print(f"No mask files found in {mask_dir}.")
#         return None, None

#     images = []
#     masks = []

#     for mask_file in mask_files:
#         # Extract the number i from the mask filename
#         match = re.search(r'NDWI_Mask_(\d+)_resized_corrupt.tif', os.path.basename(mask_file))
#         if match:
#             i = match.group(1)
#             input_file = os.path.join(input_dir, f'{i}.tif')
            
#             # Check if the corresponding input file exists
#             if os.path.exists(input_file):
#                 # Read input file
#                 with rasterio.open(input_file) as src:
#                     img = src.read(1)  # Read the first band assuming it's a single-band image
#                     images.append(img)

#                 # Read mask file
#                 with rasterio.open(mask_file) as src:
#                     msk = src.read(1)  # Read the first band assuming it's a single-band image
#                     masks.append(msk)
#             else:
#                 print(f"Corresponding input file {input_file} for mask {mask_file} not found.")

#     if not images or not masks:
#         print("No matching pairs of images and masks found.")
#         return None, None

#     return np.array(images), np.array(masks)

In [5]:
def load_data_test(input_dir, mask_dir):
    
    def numerical_sort(value):
        numbers = re.findall(r'\d+', os.path.basename(value))
        return int(numbers[-1]) if numbers else -1

    # Ensure the directories exist
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return None, None
    
    if not os.path.exists(mask_dir):
        print(f"Mask directory {mask_dir} does not exist.")
        return None, None

    # Search for .tif files in the directories
    input_files = glob.glob(os.path.join(input_dir, '*.tif'))
    mask_files = glob.glob(os.path.join(mask_dir, '*.tif'))

    print("Found input files:", input_files)
    print("Found mask files:", mask_files)

    if not input_files:
        print(f"No input files found in {input_dir}.")
        return None, None

    if not mask_files:
        print(f"No mask files found in {mask_dir}.")
        return None, None

    images = []
    masks = []

    for mask_file in mask_files:
        # Extract the number i from the mask filename
        match = re.search(r'NDWI_Mask_(\d+)_resized.tif', os.path.basename(mask_file))
        if match:
            i = match.group(1)
            input_file = os.path.join(input_dir, f'{i}.tif')
            
            # Check if the corresponding input file exists
            if os.path.exists(input_file):
                # Read input file
                with rasterio.open(input_file) as src:
                    img = src.read(1)  # Read the first band assuming it's a single-band image
                    images.append(img)

                # Read mask file
                with rasterio.open(mask_file) as src:
                    msk = src.read(1)  # Read the first band assuming it's a single-band image
                    masks.append(msk)
            else:
                print(f"Corresponding input file {input_file} for mask {mask_file} not found.")

    if not images or not masks:
        print("No matching pairs of images and masks found.")
        return None, None

    return np.array(images), np.array(masks)

In [12]:
# def load_data1(input_dir, mask_dir):
#     # Ensure the directories exist
#     if not os.path.exists(input_dir):
#         print(f"Input directory {input_dir} does not exist.")
#         return None, None
    
#     if not os.path.exists(mask_dir):
#         print(f"Mask directory {mask_dir} does not exist.")
#         return None, None

#     # Print the contents of the directories
#     print("Input directory contents:", os.listdir(input_dir))
#     print("Mask directory contents:", os.listdir(mask_dir))

#     # Search for .tif files in the directories
#     input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')))
#     mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.tif')))

#     print("Found input files:", input_files)
#     print("Found mask files:", mask_files)

#     if not input_files:
#         print(f"No input files found in {input_dir}.")
#         return None, None

#     if not mask_files:
#         print(f"No mask files found in {mask_dir}.")
#         return None, None

#     images = []
#     masks = []

#     for input_file, mask_file in zip(input_files, mask_files):
#         # Read input file
#         with rasterio.open(input_file) as src:
#             img = src.read(1)  # Read the first band assuming it's a single-band image
#             print(f"Reading image from {input_file}")
#             images.append(img)

#         # Read mask file
#         with rasterio.open(mask_file) as src:
#             msk = src.read(1)  # Read the first band assuming it's a single-band image
#             masks.append(msk)

    # return np.array(images), np.array(masks) / 255

In [13]:
def load_data_png(input_dir, mask_dir):
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.png')))
    # print(input_files)
    mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.png')))
    # print(mask_files)
    images = []
    masks = []

    for input_file, mask_file in zip(input_files, mask_files):

        image = cv2.imread(input_file)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(np.array(image_rgb))

        image = cv2.imread(mask_file)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        masks.append(np.array(image_rgb))

    # print(np.array(masks)/255)
    return np.array(images), np.array(masks)/255

In [14]:
def predict(model, images, masks, output_dir, model_type, number=-1):

    i=0
    for arbitrary_img, arbitrary_mask in zip(images, masks):

    # arbitrary_image_path = 'data_new\subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1024.tif'
    # arbitrary_image_mask = 'output_tif\subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1024_new_mask.tif'

    # # Read the arbitrary image
    # with rasterio.open(arbitrary_image_path) as src:
    #     arbitrary_img = src.read(1)  # Read the first band assuming it's a single-band image

    # with rasterio.open(arbitrary_image_mask) as src:
    #     arbitrary_mask = src.read(1)

    # Preprocess the arbitrary image if necessary (e.g., normalize, resize) to match the input dimensions of your model
    # ...

        predicted_mask = model.predict(np.expand_dims(arbitrary_img, axis=0))[0]

        # Plot the arbitrary image and the predicted mask
        plt.figure(figsize=(10, 5))

        plt.subplot(2, 2, 1)
        plt.imshow(arbitrary_img, cmap='gray')
        plt.title('Arbitrary Image')
        plt.axis('off')

        plt.subplot(2, 2, 2)
        # print(arbitrary_mask)
        print(np.min(arbitrary_mask))
        print(np.max(arbitrary_mask))
        plt.imshow(arbitrary_mask, cmap='gray')
        plt.title('Actual Mask')
        plt.axis('off')

        plt.subplot(2, 2, 3)
        plt.imshow(predicted_mask, cmap='gray')
        print("Predicted_Mask")
        print("Min value in predicted_mask:", np.min(predicted_mask))
        print("Max value in predicted_mask:", np.max(predicted_mask))
        plt.title('Predicted Mask without thresholding')
        plt.axis('off')

         # 
         #hold the predicted mask if necessary
        predicted_mask = (predicted_mask > 0.70).astype(np.uint8)

        plt.subplot(2, 2, 4)
        plt.imshow(predicted_mask, cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

        plt.savefig(f"./{output_dir}/{model_type}_{i}.jpg")
        plt.close()
        i+=1
        # break
        if (i==number):
            break

In [6]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def save_image(image, filepath):
    image = (image * 255).astype(np.uint8)  # Scale image to 0-255
    tiff.imwrite(filepath, image)

def predict1(model, mask_dir, image_dir, output_dir, model_type, number=-1):
    dice_scores = []
    iou_scores = []
    precisions = []
    recalls = []
    f1_scores = []

    os.makedirs(output_dir, exist_ok=True)

    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('_resized.tif')]
    i = 0
    print(len(mask_files))
    for mask_file in mask_files:
        # try:
        # Extract the number i from the mask file name
        print(mask_file)
        i_str = mask_file.split('_')[2]
        print(i_str)
        image_file = f"{i_str}.tif"

        mask_path = os.path.join(mask_dir, mask_file)
        image_path=os.path.join(image_dir, image_file)
        print(image_path)

        # Load the image and mask
        arbitrary_img = tiff.imread(image_path)
        arbitrary_mask = tiff.imread(mask_path)

        # Ensure the image is in the correct shape (512, 512, 1)
        if len(arbitrary_img.shape) == 2:
            arbitrary_img = np.expand_dims(arbitrary_img, axis=-1)
        elif arbitrary_img.shape[0] == 2:
            arbitrary_img = arbitrary_img[0]  # assuming you need the first channel
        
        predicted_mask = model.predict(np.expand_dims(arbitrary_img, axis=0))[0]

        # Apply thresholding to predicted mask
        predicted_mask_thresh = (predicted_mask > 0.37).astype(np.uint8)

        # Save predicted mask images
        save_image(predicted_mask, f"./{output_dir}/{model_type}_Predicted_Image_{i_str}.tif")
        save_image(predicted_mask_thresh, f"./{output_dir}/{model_type}_{i_str}.tif")

        # Plot and save the arbitrary image, actual mask, and predicted mask
        plt.figure(figsize=(10, 5))

        plt.subplot(2, 2, 1)
        plt.imshow(arbitrary_img.squeeze(), cmap='gray')
        plt.title('Arbitrary Image')
        plt.axis('off')

        plt.subplot(2, 2, 2)
        plt.imshow(arbitrary_mask, cmap='gray')
        plt.title('Actual Mask')
        plt.axis('off')

        plt.subplot(2, 2, 3)
        plt.imshow(predicted_mask.squeeze(), cmap='gray')
        plt.title('Predicted Mask without thresholding')
        plt.axis('off')

        plt.subplot(2, 2, 4)
        plt.imshow(predicted_mask_thresh.squeeze(), cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

        plt.savefig(f"./{output_dir}/{model_type}_Visualization_{i_str}.jpg")
        plt.close()

        # Calculate metrics
        dice = dice_coefficient(arbitrary_mask, predicted_mask_thresh)
        iou_score = iou(arbitrary_mask, predicted_mask_thresh)
        precision = precision_score(arbitrary_mask.flatten(), predicted_mask_thresh.flatten())
        recall = recall_score(arbitrary_mask.flatten(), predicted_mask_thresh.flatten())
        f1 = f1_score(arbitrary_mask.flatten(), predicted_mask_thresh.flatten())

        dice_scores.append(dice)
        iou_scores.append(iou_score)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    
        i += 1
        if i == number:
            break
    # Print and return the average metrics
    mean_dice = np.mean(dice_scores)
    mean_iou = np.mean(iou_scores)
    mean_precision = np.mean(precisions)
    mean_recall = np.mean(recalls)
    mean_f1 = np.mean(f1_scores)

    print(f"Mean Dice Coefficient: {mean_dice}")
    print(f"Mean IoU: {mean_iou}")
    print(f"Mean Precision: {mean_precision}")
    print(f"Mean Recall: {mean_recall}")
    print(f"Mean F1 Score: {mean_f1}")

    return mean_dice, mean_iou, mean_precision, mean_recall, mean_f1

In [16]:
input_dir = './data_new/'
mask_dir = './GEE_Masks/GEE_resized/train_gee'
output_dir = './GEE_Output/Adversarial'

# Load data
images, masks = load_data_new(input_dir, mask_dir)

# Define input shape and number of classes
images = np.expand_dims(images, axis=-1)
input_shape = images.shape[1:]

# images_train, images_test, masks_train, masks_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Create and compile the model

# for layer in model.layers:
#     print(layer.output_shape)


Found input files: ['./data_new\\0_0.tif', './data_new\\1.tif', './data_new\\10.tif', './data_new\\100.tif', './data_new\\1000.tif', './data_new\\1001.tif', './data_new\\1002.tif', './data_new\\1003.tif', './data_new\\1004.tif', './data_new\\1005.tif', './data_new\\1006.tif', './data_new\\1007.tif', './data_new\\1008.tif', './data_new\\1009.tif', './data_new\\101.tif', './data_new\\1010.tif', './data_new\\1011.tif', './data_new\\1012.tif', './data_new\\1013.tif', './data_new\\1014.tif', './data_new\\1015.tif', './data_new\\1016.tif', './data_new\\1017.tif', './data_new\\1018.tif', './data_new\\1019.tif', './data_new\\102.tif', './data_new\\1020.tif', './data_new\\1021.tif', './data_new\\1022.tif', './data_new\\1023.tif', './data_new\\1024.tif', './data_new\\1025.tif', './data_new\\1026.tif', './data_new\\1027.tif', './data_new\\1028.tif', './data_new\\1029.tif', './data_new\\103.tif', './data_new\\1030.tif', './data_new\\1031.tif', './data_new\\1032.tif', './data_new\\1033.tif', './dat

c:\Users\ADMIN\anaconda3\lib\site-packages\rasterio\__init__.py:333: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [ ]:
model = dense_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
model.fit(images, masks, validation_split=0.1, epochs=10, batch_size=2)  # Adjust epochs and batch size as needed


In [17]:

model.save('UNet_dense_GEE_adversarial_10_new_epoch')

INFO:tensorflow:Assets written to: UNet_dense_GEE_adversarial_10_new_epoch\assets


INFO:tensorflow:Assets written to: UNet_dense_GEE_adversarial_10_new_epoch\assets


In [7]:
# %%capture
# model = load_model('UNet_dense_10_epoch.h5')
# predict1(model, images, masks, output_dir, 'dense')

# import h5p
input_dir = './data_new/'
mask_dir = './GEE_Masks/GEE_resized/test_gee/'
output_dir = './GEE_Output/Adversarial/'

# Load data
# images, masks = load_data_test(input_dir, mask_dir)

# model = load_model()



In [8]:
model = tf.keras.models.load_model('UNet_dense_GEE_adversarial_10_new_epoch', compile=False)
model.compile()
predict1(model,mask_dir,input_dir, output_dir, 'dense')

632
NDWI_Mask_1000_resized.tif
1000
./data_new/1000.tif
1/1 [==============================] - 4s 4s/step
NDWI_Mask_1001_resized.tif
1001
./data_new/1001.tif
1/1 [==============================] - 0s 20ms/step
NDWI_Mask_1002_resized.tif
1002
./data_new/1002.tif
1/1 [==============================] - 0s 20ms/step
NDWI_Mask_1004_resized.tif
1004
./data_new/1004.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1005_resized.tif
1005
./data_new/1005.tif
1/1 [==============================] - 0s 10ms/step
NDWI_Mask_1007_resized.tif
1007
./data_new/1007.tif
1/1 [==============================] - 0s 18ms/step
NDWI_Mask_1008_resized.tif
1008
./data_new/1008.tif
1/1 [==============================] - 0s 10ms/step
NDWI_Mask_100_resized.tif
100
./data_new/100.tif
1/1 [==============================] - 0s 20ms/step
NDWI_Mask_1010_resized.tif
1010
./data_new/1010.tif
1/1 [==============================] - 0s 20ms/step
NDWI_Mask_1011_resized.tif
1011
./data_new/1011.tif
1/1 [========

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NDWI_Mask_1081_resized.tif
1081
./data_new/1081.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1082_resized.tif
1082
./data_new/1082.tif
1/1 [==============================] - 0s 16ms/step


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NDWI_Mask_1086_resized.tif
1086
./data_new/1086.tif
1/1 [==============================] - 0s 20ms/step


...

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NDWI_Mask_957_resized.tif
957
./data_new/957.tif
1/1 [==============================] - 0s 19ms/step
NDWI_Mask_958_resized.tif
958
./data_new/958.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_959_resized.tif
959
./data_new/959.tif
1/1 [==============================] - 0s 16ms/step
NDWI_Mask_95_resized.tif
95
./data_new/95.tif
1/1 [==============================] - 0s 16ms/step
NDWI_Mask_960_resized.tif
960
./data_new/960.tif
1/1 [==============================] - 0s 17ms/step


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NDWI_Mask_961_resized.tif
961
./data_new/961.tif
1/1 [==============================] - 0s 11ms/step
NDWI_Mask_963_resized.tif
963
./data_new/963.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_964_resized.tif
964
./data_new/964.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_965_resized.tif
965
./data_new/965.tif
1/1 [==============================] - 0s 18ms/step
NDWI_Mask_967_resized.tif
967
./data_new/967.tif
1/1 [==============================] - 0s 21ms/step
NDWI_Mask_970_resized.tif
970
./data_new/970.tif
1/1 [==============================] - 0s 10ms/step
NDWI_Mask_971_resized.tif
971
./data_new/971.tif
1/1 [==============================] - 0s 16ms/step
NDWI_Mask_972_resized.tif
972
./data_new/972.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_973_resized.tif
973
./data_new/973.tif
1/1 [==============================] - 0s 16ms/step
NDWI_Mask_974_resized.tif
974
./data_new/974.tif
1/1 [==============================] - 0s 

(0.6783806436131208,
 0.5949612386781147,
 0.6477666933213436,
 0.7528167890985689,
 0.6759217083077335)

In [ ]:
# model = residual_unet(input_shape)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# # Train the model
# model.fit(images, masks, validation_split=0.1, epochs=10, batch_size=2)
# model.save('UNet_residual_10_epoch.h5')

In [ ]:
# %%capture
# model = load_model("UNet_residual_10_epoch.h5")
# predict(model, images, masks, output_dir, 'residual')